## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-17-02-23-05 +0000,wapo,Minneapolis judge bars DHS agents from arresti...,https://www.washingtonpost.com/nation/2026/01/...
1,2026-01-17-02-22-38 +0000,nypost,LAPD is banned from using non-lethal weapons t...,https://nypost.com/2026/01/16/us-news/lapd-is-...
2,2026-01-17-02-22-13 +0000,nyt,Rick Caruso Will Not Run for L.A. Mayor or Cal...,https://www.nytimes.com/2026/01/16/us/rick-car...
3,2026-01-17-02-16-19 +0000,startribune,Twins agree with veteran catcher Victor Carati...,https://www.startribune.com/twins-sign-veteran...
4,2026-01-17-02-11-29 +0000,cbc,Chinese EVs are coming to Canada. How soon wil...,https://www.cbc.ca/news/business/chinese-evs-c...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2373/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
76,trump,65
18,ice,23
96,iran,20
233,new,15
232,president,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
122,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...,131
258,2026-01-16-12-48-22 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/16/us-news/venezuel...,124
71,2026-01-16-23-08-00 +0000,wsj,President Trump signaled that he might keep Na...,https://www.wsj.com/economy/central-banking/tr...,121
153,2026-01-16-19-25-06 +0000,wapo,Trump’s tariffs push Canada and China to strik...,https://www.washingtonpost.com/world/2026/01/1...,118
133,2026-01-16-20-23-34 +0000,nypost,Trump admin slams LA Mayor Karen Bass after sh...,https://nypost.com/2026/01/16/us-news/trump-ad...,118


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
122,131,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...
29,75,2026-01-17-00-59-04 +0000,cbc,U.S. Justice Department probing Minnesota Gov....,https://www.cbc.ca/news/world/tim-walz-jacob-f...
258,59,2026-01-16-12-48-22 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/16/us-news/venezuel...
109,52,2026-01-16-21-24-32 +0000,cbc,Here's what you need to know about the Canada-...,https://www.cbc.ca/news/politics/what-is-in-ca...
293,48,2026-01-16-10-43-00 +0000,wsj,"With Military Action on Hold, the U.S. Puts Ne...",https://www.wsj.com/world/middle-east/with-mil...
71,42,2026-01-16-23-08-00 +0000,wsj,President Trump signaled that he might keep Na...,https://www.wsj.com/economy/central-banking/tr...
317,42,2026-01-16-09-00-00 +0000,cbc,Federal government loses Emergencies Act appea...,https://www.cbc.ca/news/politics/convoy-protes...
327,40,2026-01-16-06-51-52 +0000,nypost,Ex-South Korea President Yoon sentenced to 5 y...,https://nypost.com/2026/01/16/world-news/ex-so...
46,39,2026-01-17-00-09-17 +0000,nypost,Tyler Robinson hearing takes bizarre turn as j...,https://nypost.com/2026/01/16/us-news/tyler-ro...
75,38,2026-01-16-22-59-26 +0000,latimes,"In San Francisco, Newsom rails against propose...",https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
